In [1]:
#Import of Python available libraries
import sculib

import datetime
from astropy.time import Time

#mpi = sculib.scu()
#mpi.ip = '10.96.64.10'
sim = sculib.scu()
sim.ip = '192.168.0.107'

In [2]:
'''
def csv_to_tt(csv_filename):
    import pandas as pd
    t_offset = 30
    
    df = pd.read_csv(csv_filename, names=['t', 'az', 'el'])
    t0 =  datetime.datetime.utcnow() + datetime.timedelta(seconds = t_offset) 
    t = [ t0 +  datetime.timedelta(seconds = t) for t in df['t'] ]
    return({'t': Time(t).mjd, 'az': df['az'], 'el': df['el']})
'''

"\ndef csv_to_tt(csv_filename):\n    import pandas as pd\n    t_offset = 30\n    \n    df = pd.read_csv(csv_filename, names=['t', 'az', 'el'])\n    t0 =  datetime.datetime.utcnow() + datetime.timedelta(seconds = t_offset) \n    t = [ t0 +  datetime.timedelta(seconds = t) for t in df['t'] ]\n    return({'t': Time(t).mjd, 'az': df['az'], 'el': df['el']})\n"

In [3]:
def csv_to_tt(csv_filename):
    import pandas as pd
    t_offset = 30
    
    df = pd.read_csv(csv_filename, names=['t', 'az', 'el'])
    t0 =  datetime.datetime.utcnow() + datetime.timedelta(seconds = t_offset) 
    t = [ t0 +  datetime.timedelta(seconds = t) for t in df['t'] ]
    return({'t': list(Time(t).mjd), 'az': list(df['az']), 'el': list(df['el'])})

In [4]:
def gen_ramp(az0, az1, el0, el1, start, duration):
    '''
    start and end azimuth,
    start and end elevation
    start time (datetime) and duration in seconds
    '''
    #t = [Time( start +  datetime.timedelta(seconds = t) ).mjd for t in raw_t]
    t = [ start +  datetime.timedelta(seconds = t) for t in range(0, duration + 1) ]
    az = [az0 + x * (az1-az0)/duration for x in range(len(t))]
    el = [el0 + x * (el1-el0)/duration for x in range(len(t))]
    return({'t': Time(t).mjd, 'az': az, 'el': el})

def ramp_table(azimuth, elevation):
    angle_duration_pair = [ 
    [0.0001, 120],
    [0.001, 120],
    [0.01, 120],
    [0.1, 120],
    [0.3, 120],
    [0.5, 120],
    [1, 60] ]
    #this portion generates the track table for all the sweeps at a particular azimuth/elevation
    sign = 1
    body = '' #body is the track table string, starts empty
    offset = 30 #time offset to start of track table
    t_prep = 30 #time preparation between sweeps
    old_aoff = 0 #need to keep track of end position of previous sweep
    t =  datetime.datetime.utcnow() + datetime.timedelta(seconds = offset) 
    for angle_off, duration in angle_duration_pair:
        print("\t Angle offset: {} Duration: {}".format(angle_off, 2*duration))       
        #this generates track table segment from end of previous sweep to start of next sweep
        body += sim.format_body(**gen_ramp(azimuth-sign*old_aoff, azimuth-sign*angle_off, elevation, elevation, t, t_prep))
        t = t + datetime.timedelta(seconds = t_prep + 1)
        #this generates track table segment for the sweep itself
        body += sim.format_body(**gen_ramp(azimuth-sign*angle_off, azimuth+sign*angle_off, elevation, elevation, t, 2*duration))
        t = t + datetime.timedelta(seconds = 2*duration + 1)
        #we change sign every time to sweep from opposite ends, for efficiency
        sign = -sign
        #we remember the previous end of the sweep to prepare for next sweep
        old_aoff = angle_off
    return(body)

def mpi_t682(scu):
    print('Test 6.8.2')

    #Revisions:
    #0.1 initial write - could do with some kat sensor retrieval
    #0.2 rewrite for sculib

    #Common setup
    config_name = 'HN_TILT_TEST'

    
    azimuths = [-180, -120, -60, 0, 60, 120 ,180]
    az_start = -180
    az_speed = 3
    
    elevations = [15, 52, 85]
    el_start = 15 # 
    el_speed  = 1 #  slew rate of 1 should be possible after retrofit


    
    wait1 =1 
    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200
    scu.debug = False
    #debug test only remove these
    #'''
    azimuths = [0]
    az_start = 0
    az_speed = 3
    
    elevations = [19]
    el_start = 19 # 
    el_speed  = 1 #  slew rate of 1 should be possible after retrofit

    wait1 =1 
    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200
    scu.debug = True
    
    #'''
  
    '''
    Track table with position coordinates at 1 second intervals to represent a ramp with the following rates and durations:
    +0.0001°/sec for 120 sec, -0.0001°/sec for 120 sec
    +0.001°/sec for 120 sec, -0.001°/sec for 120 sec
    +0.01°/sec for 120 sec, -0.01°/sec for 120 sec
    +0.1°/sec for 120 sec, -0.1°/sec for 120 sec
    +0.3°/sec for 120 sec, -0.3°/sec for 120 sec
    +0.5°/sec for 120 sec, -0.5°/sec for 120 sec
    +1°/sec for 60 sec, -1°/sec for 60 sec

    1) Manually move the Dish to 15° Elevation, -180° Azimuth and de-activate all axes
    2) Activate the Azimuth and Elevation axis
    • wait for 15 seconds to complete the activation process
    3) Command Elevation to 15° at 1°/sec and Azimuth to -180° at 3°/sec and (“Move to Absolute Position”)
    • wait for 70 seconds to complete the move and to stabilise
    4) Upload the Track Table with test profile defined below to the SCU
    • wait for 5 seconds
    5) Start the ACU data recording on the SCU using logging configuration TBD
    • wait for 5 seconds
    6) Command Azimuth to start tracking the loaded table
    • wait for 1080 seconds to complete the track table
    7) Stop the ACU data recording
    8) Repeat steps 3) to 7) for Elevation angles of 52° and 85°
    9) De-activate all axis
    10) Repeat steps 2) to 9) with Azimuth angles of 120°, 60°, 0°, 60°, 120° and 180°
    11) Extract the data on the SCU @ 5Hz sampling
    12) Rename the files to TBD (eg. 6p8p2_i.csv)
    13) Download the MeerKAT weather station data for the test period
    '''

    #Step 1
    #Manual move dish to  15° Elevation, -180° Azimuth
    scu.activate_dmc()
    scu.wait_duration(wait10*2) 
    
    print("Manual move to az: {} and el: {}".format(az_start, el_start))
    az_angle = az_start
    scu.abs_azimuth(az_angle, az_speed)
    scu.wait_duration(wait10*3)

    el_angle = el_start
    scu.abs_elevation(el_angle, el_speed)
    scu.wait_duration(wait10*3)     #scu.reset_dmc()

    #Step 2 lets just skip an activate and deactive

    #Step 3 - command 
  
    #Step 4 - Load track table
    for azimuth in azimuths:
        print("\t Az center: {} ".format(azimuth)) 
        az_angle = azimuth
        scu.abs_azimuth(az_angle, az_speed)
        scu.wait_duration(wait10*3)
        for elevation in elevations:
            el_angle = elevation
            scu.abs_elevation(el_angle, el_speed)
            scu.wait_duration(wait10*3) 
            print("\t El center: {} ".format(elevation))
            #Step 5 - start ACU recording on SCU
            if scu.logger_state() == 'RECORDING':
                print('WARNING, already recording - attempting to stop and start a fresh logger')
                scu.stop_logger()  
                scu.wait_duration(wait10)
            if scu.logger_state() == 'STOPPED':
                print('Starting logger with config: {}'.format(config_name))
                scu.start_logger(config_name)
                scu.wait_duration(wait10)

            #fire and forget the track table generated for this azimuth and elevation
            scu.acu_ska_track( scu.format_body(**csv_to_tt('voorbeeld.csv')) )
            scu.wait_duration(wait10*3)  #time offset is 30s as defined by offset abovet
                
            #Step 6 - Command Azimuth to start tracking the loaded table
            scu.wait_duration(100+841)  #It takes 30 minutes for all the sweeps             
                
            #Step 7) Stop the ACU data recording
            scu.stop_logger()
            scu.wait_duration(wait10)
            print(scu.logger_state())

            #Step 11) Extract the data on the SCU @ 5Hz sampling
            #Step 12) Rename the files to TBD (eg. 6p8p2_i.csv)
            print("Rename the file to TBD (eg. 6p8p2_i.csv)")
            scu.save_session('6p8p2_i_az_' + str(azimuth) + '_el_' + str(elevation), interval_ms = sampling)

        #Step 8) Repeat steps 3) to 7) for Elevation angles of 52° and 85°
        #Step 9) De-activate all axis ? IGNORED here done at end.
        #Step 10) Repeat steps 2) to 9) with Azimuth angles of 120°, 60°, 0°, 60°, 120° and 180°
        
        #13) Download the MeerKAT weather station data for the test period

    #Step x) De-activate all axis
    scu.deactivate_dmc()
    scu.wait_duration(wait10*2) 
    

In [5]:
#mpi_t682(mpi)
mpi_t682(sim)


Test 6.8.2
activate dmc
***Feedback: http://192.168.0.107:8080/devices/command b'{"path": "acu.dish_management_controller.activate"}'
OK 200
***Text returned:
{"uuid":"1c31ff1a-2bf7-402c-b6e4-12a8ac60df6e"}

wait for 20.0s done *
Manual move to az: 0 and el: 19
abs az: 0.0000 vel: 3.0000
***Feedback: http://192.168.0.107:8080/devices/command b'{"path": "acu.azimuth.slew_to_abs_pos", "params": {"new_axis_absolute_position_set_point": 0, "new_axis_speed_set_point_for_this_move": 3}}'
OK 200
***Text returned:
{"uuid":"819e477a-092d-4006-bdac-82e36d91b60e"}

wait for 30.0s done *
abs el: 19.0000 vel: 1.0000
***Feedback: http://192.168.0.107:8080/devices/command b'{"path": "acu.elevation.slew_to_abs_pos", "params": {"new_axis_absolute_position_set_point": 19, "new_axis_speed_set_point_for_this_move": 1}}'
OK 200
***Text returned:
{"uuid":"ba40f3b9-c2bc-4e3d-ac1c-a82d1f5cc2cd"}

wait for 30.0s done *
	 Az center: 0 
abs az: 0.0000 vel: 3.0000
***Feedback: http://192.168.0.107:8080/devices/co

wait for 30.0s done *
wait for 941.0s done *
stop logger
***Feedback: http://192.168.0.107:8080/datalogging/stop b'{}'
OK 200
***Text returned:
{"uuid":null,"startTime":"2020-11-16T10:22:30.403574800Z","stopTime":"2020-11-16T10:38:52.091079400Z","duration":"00:16:21","configId":25,"fullFilePath":".\\datalogging\\2020\\11\\16___10_22_30","name":"HN_TILT_TEST"}

wait for 10.0s done *
logger state 
***Feedback: http://192.168.0.107:8080/datalogging/currentState None
OK 200
***Text returned:
{"state":"STOPPED"}

STOPPED
Rename the file to TBD (eg. 6p8p2_i.csv)
Attempt export and save of session: last at rate 200 ms
logger sessions 
***Feedback: http://192.168.0.107:8080/datalogging/sessions None
OK 200
***Text returned:
[{"uuid":1,"startTime":"2020-08-24T12:28:09.219205300Z","stopTime":"2020-08-24T12:29:10.925214900Z","duration":"00:01:01","configId":3,"fullFilePath":".\\datalogging\\2020\\8\\24___12_28_09","name":"index_logger.txt"},{"uuid":2,"startTime":"2020-08-24T12:32:20.466192700Z","

***Feedback: http://192.168.0.107:8080/datalogging/exportSession?id=378&interval_ms=200 None
OK 200
***Text returned:
StartTime;2020-11-16T10:22:30.403574800Z
EndTime;2020-11-16T10:38:52.091079400Z
;acu.time.act_time_source;acu.time.internal_time;acu.time.external_ptp;acu.general_management_and_controller.state;acu.general_management_and_controller.feed_indexer_pos;acu.azimuth.state;acu.azimuth.p_set;acu.azimuth.p_act;acu.azimuth.v_act;acu.elevation.state;acu.elevation.p_set;acu.elevation.p_act;acu.elevation.v_act;acu.general_management_and_controller.act_power_consum;acu.general_management_and_controller.power_factor;acu.general_management_and_controller.voltage_phase_1;acu.general_management_and_controller.voltage_phase_2;acu.general_management_and_controller.voltage_phase_3;acu.general_management_and_controller.current_phase_1;acu.general_management_and_controller.current_phase_2;acu.general_management_and_controller.current_phase_3;acu.pointing.act_amb_temp_1;acu.pointing.act_amb_t

wait for 20.0s done *


In [6]:
#sim.save_session('6p8p2_i_az_' + '000' + '_el_' + '19', interval_ms = 200)

In [7]:

#csv_to_tt('voorbeeld.csv')
#sim.format_body(**csv_to_tt('voorbeeld.csv'))
#sim.acu_ska_track( sim.format_body(**csv_to_tt('voorbeeld.csv')) )
#sim.acu_ska_track(sim.format_body(**csv_to_tt('voorbeeld.csv')) )
#print(sim.format_body(**csv_to_tt('voorbeeld.csv')) )
#print(csv_to_tt('voorbeeld.csv')) 
